In [1]:
import numpy as np
from utils import get_img_by_bbox

In [2]:
# get_img_by_bbox("31.141762466358795, -7.9263350168656075","31.12902260273618, -7.908643158195691")

In [3]:
import requests
from geopy.geocoders import Nominatim

def fetch_villages(xa, ya, xb, yb):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node["place"="village"]({ya},{xa},{yb},{xb});
      way["place"="village"]({ya},{xa},{yb},{xb});
      relation["place"="village"]({ya},{xa},{yb},{xb});
    );
    out center;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    geolocator = Nominatim(user_agent="geoapianasio")

    villages = []
    print(len(data['elements']))
    # for element in data['elements']:
    #     print(element)
    #     # Extract the center of the village if it's a way or relation
    #     lat = element.get('center', {}).get('lat', element.get('lat'))
    #     lon = element.get('center', {}).get('lon', element.get('lon'))
    #     name = element.get('tags', {}).get('name', 'Unknown')

    #     # Get altitude using geopy
    #     location = geolocator.reverse((lat, lon), exactly_one=True)
    #     altitude = location.raw.get('altitude', 'Unknown')

    #     village_info = {
    #         'name': name,
    #         'latitude': lat,
    #         'longitude': lon,
    #         'altitude': altitude
    #     }
    #     print("-----------------------------------------")
    #     print(village_info)
    #     villages.append(village_info)

    return data


# # Morocco bounding box coordinates
# xa, ya, xb, yb = -13.172, 27.662, -1.009, 35.922
# villages = fetch_villages(xa, ya, xb, yb)

# xa, ya, xb, yb =  31.48101206796215 , -9.084209313190824 ,31.0695654713028,  -7.440869475147751
# xa, ya, xb, yb = 31.34491591874058, -8.672982822954419 , 30.92706411399971, -7.486593538320688
xa, ya, xb, yb = -13.172, 27.662, -1.009, 35.922
villages = fetch_villages(xa, ya, xb, yb)


7354


In [4]:
all_ma_villages = villages["elements"]
clean_villages = [] 
error = 0
for vil in all_ma_villages : 
    try: 
        clean_villages.append({
            "id":vil["id"],
            "lat":vil["lat"],
            "lon":vil["lon"],
            "name":vil["tags"].get("name","no name")
        })
    except : 
        error += 1
        

import pandas as pd
# df = pd.DataFrame(all_ma_villages)
# df = pd.json_normalize(all_ma_villages, 'tags', ['type', 'id', 'lat', 'lon'])
df = pd.DataFrame(clean_villages)

df


,id,lat,lon,name
0,119865925,31.660092,-7.917141,Bab Atlas ⴱⴰⴱ ⴰⵟⵍⴰⵙ
1,256921015,29.499957,-10.062260,Aguni ⴰⴳⵓⵏⵉ أكوني
2,256924648,29.622114,-10.006425,Id Hammouch ⵉⴷ ⵃⵎⵎⵓⵛ إد حموش
3,256925195,29.731195,-9.911325,Sidi Bounouar سيدي بونوار
4,256932131,29.726087,-9.769184,Tadouarte ⵜⴰⴹⵡⵡⴰⵔⵜ تدوارت
...,...,...,...,...
7224,11854270239,29.477824,-5.591562,Tindjoub
7225,11881782958,30.606756,-8.404584,Ait Daoud أيت داود
7226,11891747545,30.429526,-7.899658,Aouerste أورست
7227,11891807969,30.518244,-7.794301,Miggarn مكــارن


In [5]:
filtered_df = df[df['name'].str.contains('oughlad', case=False)]
filtered_df

,id,lat,lon,name
5721,6208702034,31.200799,-7.957374,Imi Oughlad


In [6]:
# 31.17966683778746, -7.946329868456538


df[(df.index<5800) & (df.index>5700)]

,id,lat,lon,name
5701,6168553830,34.743889,-4.381304,no name
5702,6168554470,34.751396,-4.400654,no name
5703,6173687416,30.284011,-7.074873,Taourirt n'Ouznag
5704,6173687417,30.310157,-7.128839,Imi n'Oumagha
5705,6173687418,30.312204,-7.103358,Timzoughine
...,...,...,...,...
5795,7266376374,30.050442,-5.513164,Ksar Ait Zemrou
5796,7266378946,30.026854,-5.513639,Ksar Oulad Youssef
5797,7266418809,29.922814,-5.509693,Ksar Lkbir
5798,7266445003,30.648909,-6.144481,Ksar Taguersift


In [7]:
import math



def coordinates_of_square_centered_at(coord, radius):
    """
    Calculate the coordinates (up corner, b corner) of a square centered around the input geo coordinate
    with a given radius in kilometers.
    
    Parameters:
    coord (tuple): A tuple containing the latitude and longitude of the center point (in degrees).
    radius (float): The radius of the square (in kilometers).
    
    Returns:
    tuple: A tuple containing the coordinates of the upper-left and bottom-right corners of the square.
    """
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat, lon = math.radians(coord[0]), math.radians(coord[1])
    
    # Calculate the change in latitude and longitude due to the radius
    delta_lat = radius / R * (180.0 / math.pi)
    delta_lon = radius / (R * math.cos(lat)) * (180.0 / math.pi)
    
    # Calculate the coordinates of the upper-left corner of the square
    up_corner = (coord[0] + delta_lat, coord[1] - delta_lon)
    
    # Calculate the coordinates of the bottom-right corner of the square
    b_corner = (coord[0] - delta_lat, coord[1] + delta_lon)
    
    return up_corner, b_corner

# # Example usage:
# center_coord = (40.7128, -74.0060)  # New York City coordinates (latitude, longitude)
# radius_km = 10  # Radius in kilometers

# up_corner, b_corner = coordinates_of_square_centered_at(center_coord, radius_km)
# print("Upper-left corner:", up_corner)
# print("Bottom-right corner:", b_corner)


# # Example usage:
# center_coord = (40.7128, -74.0060)  # New York City coordinates (latitude, longitude)
# radius_km = 10  # Radius in kilometers

# up_corner, b_corner = coordinates_of_square_centered_at(center_coord, radius_km)



In [8]:
# search by name 
query = "Imlil ⵉⵎⵍⵉⵍ "
filtered_df = df[df['name'].str.contains(query, case=False)]
print()
print(f"results for query {query}", f" - count = {len(filtered_df)} records")
print("-----------------")
print(filtered_df)
print("-----------------")
resp_idx = 1

meta = filtered_df.to_dict(orient='records')[resp_idx]
# print(meta)


print()
print(" genereting bbox cordinates  ... ")
up_corner, b_corner = coordinates_of_square_centered_at(coord = (meta["lat"],meta["lon"]),
                                                        radius= 1)



bbox_up_cors_str = ",".join([str(i) for i in up_corner]) 
bbox_bt_cors_str =  ",".join([str(i) for i in b_corner])

# print(up_corner, b_corner)
print()
print(" genereting img for bbox  ... ")
get_img_by_bbox(bbox_up_cors_str,bbox_bt_cors_str,zoom = 17)


results for query Imlil ⵉⵎⵍⵉⵍ   - count = 2 records
-----------------
              id        lat       lon               name
2603  2272230717  31.755427 -7.012107  Imlil ⵉⵎⵍⵉⵍ إمليل
5396  4076213431  31.143889 -7.921819  Imlil ⵉⵎⵍⵉⵍ إمليل
-----------------

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img__20240530211616.png


In [9]:
df

,id,lat,lon,name
0,119865925,31.660092,-7.917141,Bab Atlas ⴱⴰⴱ ⴰⵟⵍⴰⵙ
1,256921015,29.499957,-10.062260,Aguni ⴰⴳⵓⵏⵉ أكوني
2,256924648,29.622114,-10.006425,Id Hammouch ⵉⴷ ⵃⵎⵎⵓⵛ إد حموش
3,256925195,29.731195,-9.911325,Sidi Bounouar سيدي بونوار
4,256932131,29.726087,-9.769184,Tadouarte ⵜⴰⴹⵡⵡⴰⵔⵜ تدوارت
...,...,...,...,...
7224,11854270239,29.477824,-5.591562,Tindjoub
7225,11881782958,30.606756,-8.404584,Ait Daoud أيت داود
7226,11891747545,30.429526,-7.899658,Aouerste أورست
7227,11891807969,30.518244,-7.794301,Miggarn مكــارن


In [11]:
c = 0
tresh  = 100
for v_meta in df.values: 
    c +=1
    idx,lat,lon,name = v_meta
    print(lat,lon,name)
    print()
    print(" genereting bbox cordinates  ... ")
    up_corner, b_corner = coordinates_of_square_centered_at(coord = (meta["lat"],meta["lon"]),
                                                            radius= 1)



    bbox_up_cors_str = ",".join([str(i) for i in up_corner]) 
    bbox_bt_cors_str =  ",".join([str(i) for i in b_corner])

    # print(up_corner, b_corner)
    print()
    print(" genereting img for bbox  ... ")
    get_img_by_bbox(bbox_up_cors_str,bbox_bt_cors_str,zoom = 17,village_name=f"{idx}_{name}")
    if c>tresh:
        break


31.6600915 -7.9171412 Bab Atlas ⴱⴰⴱ ⴰⵟⵍⴰⵙ

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img_119865925_Bab Atlas ⴱⴰⴱ ⴰⵟⵍⴰⵙ_20240530211657.png
29.4999574 -10.0622603 Aguni ⴰⴳⵓⵏⵉ أكوني

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img_256921015_Aguni ⴰⴳⵓⵏⵉ أكوني_20240530211658.png
29.6221144 -10.006425 Id Hammouch ⵉⴷ ⵃⵎⵎⵓⵛ إد حموش

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img_256924648_Id Hammouch ⵉⴷ ⵃⵎⵎⵓⵛ إد حموش_20240530211700.png
29.7311954 -9.9113246 Sidi Bounouar سيدي بونوار

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img_256925195_Sidi Bounouar سيدي بونوار_20240530211702.png
29.7260868 -9.7691843 Tadouarte ⵜⴰⴹⵡⵡⴰⵔⵜ تدوارت

 genereting bbox cordinates  ... 

 genereting img for bbox  ... 
(1958, 1959, 3)
Saved as img_256932131_Tadouarte ⵜⴰⴹⵡⵡⴰⵔⵜ تدوارت_20240530211703.png
31.225888 -7.675342 Sitti 